In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
name_list = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "label"]

In [3]:
df_train = pd.read_csv('adult.data', names = name_list)
df_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
df_train['label'].value_counts()

 <=50K    24720
 >50K      7841
Name: label, dtype: int64

In [4]:
df_test = pd.read_csv('adult.test', names = name_list)
df_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [6]:
df_test["education-num"].value_counts()

9     5283
10    3587
13    2670
14     934
11     679
7      637
12     534
6      456
4      309
15     258
5      242
8      224
16     181
3      176
2       79
1       32
Name: education-num, dtype: int64

In [9]:
from sklearn import preprocessing

df_train_shape = df_train.shape
Xtrain = np.zeros([df_train_shape[0], df_train_shape[1]-1])
ytrain = np.zeros([df_train_shape[0], 1])

df_test_shape = df_test.shape
Xtest = np.zeros([df_test_shape[0], df_test_shape[1]-1])
ytest = np.zeros([df_test_shape[0], 1])

for i in range(df_train_shape[1]-2):
    if df_train.dtypes.index[i] == 'fnlwgt' or df_train.dtypes.index[i] == 'age' or df_train.dtypes.index[i] == 'education-num' or df_train.dtypes.index[i] == 'capital-gain' or df_train.dtypes.index[i] == 'capital-loss' or df_train.dtypes.index[i] == 'hours-per-week':
        df_inp = df_train.iloc[:, i]
        df_enc = df_inp
        Xtrain[:, i] = df_enc
        
        df_inp = df_test.iloc[:, i]
        df_enc = df_inp
        Xtest[:, i] = df_enc
    else:
        le = preprocessing.LabelEncoder()
        df_inp = df_train.iloc[:, i]
        df_enc = le.fit_transform(df_inp)
        Xtrain[:, i] = df_enc
        
        df_inp = df_test.iloc[:, i]
        df_enc = le.fit_transform(df_inp)
        Xtest[:, i] = df_enc

le = preprocessing.LabelEncoder()
df_inp = df_train.iloc[:, df_train_shape[1]-1]
df_enc = le.fit_transform(df_inp)
ytrain[:, 0] = df_enc

df_inp = df_test.iloc[:, df_test_shape[1]-1]
df_enc = le.fit_transform(df_inp)
ytest[:, 0] = df_enc

XtrainOriginal = Xtrain.copy()
ytrainOriginal = ytrain.copy()

In [10]:
XtrainOriginal[0]

array([3.9000e+01, 7.0000e+00, 7.7516e+04, 9.0000e+00, 1.3000e+01,
       4.0000e+00, 1.0000e+00, 1.0000e+00, 4.0000e+00, 1.0000e+00,
       2.1740e+03, 0.0000e+00, 4.0000e+01, 0.0000e+00])

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


train_acc_list = np.zeros((30,1))
test_acc_list = np.zeros((30,1))
for iter in range(1000):
    for m in range(1,30):
        print('iter: ' + str(iter) + ', m: ' + str(m))

        inds = np.random.choice(range(XtrainOriginal.shape[0]), m, replace=False)
        Xtrain = XtrainOriginal[inds]
        ytrain = ytrainOriginal[inds]

        clf = GaussianNB(priors=None)
        clf.fit(Xtrain, np.ravel(ytrain))

        ypred_train = clf.predict(Xtrain)
        train_acc_list[m] = train_acc_list[m] + mae(ytrain, ypred_train)

        ypred = clf.predict(Xtest)
        test_acc_list[m] = test_acc_list[m] + mae(ytest,ypred)
        
train_acc_list = train_acc_list/1000.0
test_acc_list = test_acc_list/1000.0

iter: 0, m: 1
iter: 0, m: 2
iter: 0, m: 3
iter: 0, m: 4


C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 

iter: 0, m: 5
iter: 0, m: 6
iter: 0, m: 7
iter: 0, m: 8
iter: 0, m: 9
iter: 0, m: 10
iter: 0, m: 11
iter: 0, m: 12
iter: 0, m: 13
iter: 0, m: 14
iter: 0, m: 15
iter: 0, m: 16
iter: 0, m: 17
iter: 0, m: 18
iter: 0, m: 19
iter: 0, m: 20
iter: 0, m: 21
iter: 0, m: 22
iter: 0, m: 23
iter: 0, m: 24
iter: 0, m: 25
iter: 0, m: 26
iter: 0, m: 27
iter: 0, m: 28
iter: 0, m: 29
iter: 1, m: 1


C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 

iter: 1, m: 2
iter: 1, m: 3
iter: 1, m: 4
iter: 1, m: 5
iter: 1, m: 6
iter: 1, m: 7
iter: 1, m: 8
iter: 1, m: 9
iter: 1, m: 10
iter: 1, m: 11
iter: 1, m: 12
iter: 1, m: 13
iter: 1, m: 14
iter: 1, m: 15
iter: 1, m: 16
iter: 1, m: 17
iter: 1, m: 18
iter: 1, m: 19
iter: 1, m: 20
iter: 1, m: 21
iter: 1, m: 22
iter: 1, m: 23
iter: 1, m: 24
iter: 1, m: 25
iter: 1, m: 26
iter: 1, m: 27
iter: 1, m: 28
iter: 1, m: 29
iter: 2, m: 1
iter: 2, m: 2
iter: 2, m: 3
iter: 2, m: 4
iter: 2, m: 5
iter: 2, m: 6


...

iter: 997, m: 7
iter: 997, m: 8
iter: 997, m: 9
iter: 997, m: 10
iter: 997, m: 11
iter: 997, m: 12
iter: 997, m: 13
iter: 997, m: 14
iter: 997, m: 15
iter: 997, m: 16
iter: 997, m: 17
iter: 997, m: 18
iter: 997, m: 19
iter: 997, m: 20
iter: 997, m: 21
iter: 997, m: 22
iter: 997, m: 23
iter: 997, m: 24
iter: 997, m: 25
iter: 997, m: 26
iter: 997, m: 27
iter: 997, m: 28
iter: 997, m: 29
iter: 998, m: 1
iter: 998, m: 2
iter: 998, m: 3
iter: 998, m: 4


C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 

iter: 998, m: 5
iter: 998, m: 6
iter: 998, m: 7
iter: 998, m: 8
iter: 998, m: 9
iter: 998, m: 10
iter: 998, m: 11
iter: 998, m: 12
iter: 998, m: 13
iter: 998, m: 14
iter: 998, m: 15
iter: 998, m: 16
iter: 998, m: 17
iter: 998, m: 18
iter: 998, m: 19
iter: 998, m: 20
iter: 998, m: 21
iter: 998, m: 22
iter: 998, m: 23
iter: 998, m: 24
iter: 998, m: 25
iter: 998, m: 26
iter: 998, m: 27
iter: 998, m: 28
iter: 998, m: 29
iter: 999, m: 1


C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:436: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
C:\Users\SaiTeja\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\naive_bayes.py:438: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 

iter: 999, m: 2
iter: 999, m: 3
iter: 999, m: 4
iter: 999, m: 5
iter: 999, m: 6
iter: 999, m: 7
iter: 999, m: 8
iter: 999, m: 9
iter: 999, m: 10
iter: 999, m: 11
iter: 999, m: 12
iter: 999, m: 13
iter: 999, m: 14
iter: 999, m: 15
iter: 999, m: 16
iter: 999, m: 17
iter: 999, m: 18
iter: 999, m: 19
iter: 999, m: 20
iter: 999, m: 21
iter: 999, m: 22
iter: 999, m: 23
iter: 999, m: 24
iter: 999, m: 25
iter: 999, m: 26
iter: 999, m: 27
iter: 999, m: 28
iter: 999, m: 29


In [7]:
test_acc_list

array([[0.        ],
       [0.63083177],
       [0.63751391],
       [0.6817556 ],
       [0.69558995],
       [0.71398587],
       [0.71468006],
       [0.72348296],
       [0.72299877],
       [0.71989939],
       [0.72263061],
       [0.71465137],
       [0.71808335],
       [0.70829151],
       [0.71766237],
       [0.71367797],
       [0.70192531],
       [0.7065227 ],
       [0.70527486],
       [0.70310392],
       [0.70841355],
       [0.69472023],
       [0.7128443 ],
       [0.70860709],
       [0.69907659],
       [0.70723045],
       [0.70532129],
       [0.70147092],
       [0.70659732],
       [0.70318082]])

In [ ]:
from sklearn.linear_model import LogisticRegression

train_acc_list_lr = np.zeros((30,1))
test_acc_list_lr = np.zeros((30,1))
for iter in range(1000):
    for m in range(2,30):
        print('iter: ' + str(iter) + ', m: ' + str(m))

        inds = np.random.choice(range(XtrainOriginal.shape[0]), m, replace=False)
        Xtrain = XtrainOriginal[inds]
        ytrain = ytrainOriginal[inds]
        z = np.ravel(ytrain)
        l = len(set(z))
        while(l < 2):
            inds = np.random.choice(range(XtrainOriginal.shape[0]), m, replace=False)
            Xtrain = XtrainOriginal[inds]
            ytrain = ytrainOriginal[inds]
            z = np.ravel(ytrain)
            l = len(set(z))

        
        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(Xtrain, np.ravel(ytrain))
#         clf.fit(Xtrain, np.ravel(ytrain))

        ypred_train = clf.predict(Xtrain)
        train_acc_list_lr[m] = train_acc_list_lr[m] + mse(ytrain, ypred_train)

        ypred = clf.predict(Xtest)
        test_acc_list_lr[m] = test_acc_list_lr[m] + mse(ytest,ypred)
        
train_acc_list_lr = train_acc_list_lr/1000.0
test_acc_list_lr = test_acc_list_lr/1000.0

In [ ]:
test_acc_list_lr

In [ ]:
plt.plot(range(2,30),test_acc_list[2:],label = "Naive Bayes")
plt.plot(range(2,30),test_acc_list_lr[2:],label = "Logistic Regression")
plt.legend()
plt.savefig("adult30_1000.png")
# plt.xlim(2, 30) 

In [ ]:
pwd